import sys
#sys.path
sys.path.append("/opt/homebrew/Caskroom/miniforge/base/envs/sunpy/lib/python3.11/site-packages/")

In [2]:
import glob,datetime
import sunpy.coordinates
import numpy as np
from sunpy.coordinates import spice,HeliographicStonyhurst, HeliographicCarrington
import matplotlib.pyplot as plt
import helpers as h
kernel_files = glob.glob("./spice_data/*.bsp")
spice.initialize(kernel_files)
spice.install_frame('IAU_SUN')
plt.style.use("dark_background")

INFO: Installing IAU_SUN PCK frame (10010) as 'spice_IAU_SUN' [sunpy.coordinates.spice]
INFO: Creating ICRF frame with SUN (10) origin [sunpy.coordinates.spice]


In [3]:
import pyspedas, pytplot
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import getpass
import astropy.units as u

# Custom
import helpers as h

### sunpy ###
from sunpy.net import Fido, attrs as a
import sunpy_soar

#plt.style.use("dark_background")

28-May-25 16:37:21: /Users/lachmed/pyspedas/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



In [4]:
### Parker Solar Probe Plasma Data
# SPC
dt_start = datetime.datetime(2024, 5, 1)
dt_end = datetime.datetime(2024, 5, 31)

#usr = getpass.getuser()
#pass_swp = getpass.getpass("Enter SWEAP Password:")
#pass_fld = getpass.getpass("Enter FIELDS Password:")


psp_spc = pyspedas.psp.spc([dt_start,dt_end],
                                       notplot=True,    
                                       datatype="l3i", 
                                       level="L3")
                                       #username=usr,
                                       #password=pass_swp,)

#Density
psp_spc_Dens=psp_spc['psp_spc_np_fit']
parker_n_dt = pd.to_datetime(psp_spc_Dens['x']).to_pydatetime() # Datetimes
parker_n_ts = np.array([dt.timestamp() for dt in parker_n_dt]) # Unix Timestamp (float)
parker_n = psp_spc_Dens['y'] 
#parker_n

#speed
psp_spc_VEL_RTN_SUN  = psp_spc['psp_spc_vp_fit_RTN']
parker_vr_dt = pd.to_datetime(psp_spc_VEL_RTN_SUN['x']).to_pydatetime() # Datetimes
parker_vr_ts = np.array([dt.timestamp() for dt in parker_vr_dt]) # Unix Timestamp (float)
parker_vr = psp_spc_VEL_RTN_SUN['y'][:,0]

#Temp
psp_spc_Temp  = psp_spc['psp_spc_wp_fit']
parker_w_dt = pd.to_datetime(psp_spc_Temp['x']).to_pydatetime() # Datetimes
parker_w_ts = np.array([dt.timestamp() for dt in parker_w_dt]) # Unix Timestamp (float)
parker_w = psp_spc_Temp['y']

28-May-25 16:37:25: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2024/
28-May-25 16:37:25: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240501_v02.cdf to psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240501_v02.cdf


Using LEVEL=L3


28-May-25 16:37:25: Download complete: psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240501_v02.cdf
28-May-25 16:37:25: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240502_v02.cdf to psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240502_v02.cdf
28-May-25 16:37:25: Download complete: psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240502_v02.cdf
28-May-25 16:37:25: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240503_v02.cdf to psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240503_v02.cdf
28-May-25 16:37:25: Download complete: psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240503_v02.cdf
28-May-25 16:37:25: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240504_v02.cdf to psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240504_v02.cdf
28-May-25 16:37:25: Download complete: psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240504_v02.cdf
28-May

In [5]:
# Generate Positions for Parker Solar Probe plasma data
parker_trajectory_inertial_Vr = spice.get_body('SPP', parker_vr_dt)
parker_trajectory_carrington_Vr = parker_trajectory_inertial_Vr.transform_to(HeliographicCarrington(observer="self"))
parker_trajectory_inertial_Vr.representation_type = "spherical"
parker_trajectory_carrington_Vr.representation_type = "spherical"